# Libs

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('titanic.csv')
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

# hw 1

In [ ]:
# Какое количество мужчин и женщин ехало на корабле? В качестве ответа приведите два числа через пробел.

# Какой части пассажиров удалось выжить? Посчитайте долю выживших пассажиров. 
# Ответ приведите в процентах (число в интервале от 0 до 100, знак процента не нужен), округлив до двух знаков.

# Какую долю пассажиры первого класса составляли среди всех пассажиров? Ответ приведите в процентах 
# (число в интервале от 0 до 100, знак процента не нужен), округлив до двух знаков.

# Какого возраста были пассажиры? Посчитайте среднее и медиану возраста пассажиров. 
# В качестве ответа приведите два числа через пробел.

# Коррелируют ли число братьев/сестер/супругов с числом родителей/детей? 
# Посчитайте корреляцию Пирсона между признаками SibSp и Parch.

# Какое самое популярное женское имя на корабле? Извлеките из полного имени пассажира (колонка Name) 
# его личное имя (First Name). Это задание — типичный пример того, с чем сталкивается специалист по анализу данных. 
# Данные очень разнородные и шумные, но из них требуется извлечь необходимую информацию. 
# Попробуйте вручную разобрать несколько значений столбца Name и выработать правило для извлечения имен, 
# а также разделения их на женские и мужские.

In [7]:
df.Sex.value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [24]:
total = df.PassengerId.count()
surv = df[df.Survived==1].Survived.count()
survived = round((surv/total)*100,2)
survived

38.38

In [23]:
first_class = df[df.Pclass==1].Pclass.count()
first_class = round((first_class/total)*100,2)
first_class

24.24

In [124]:
age = round(df.Age.mean(),2)
age_p50 = round(df.Age.median(),2)
age, age_p50

(29.7, 28.0)

In [36]:
correl = df.SibSp.corr(df.Parch, method='pearson').round(2)
correl

0.41

In [123]:
names = df.Name.str.split(',').str[1]
contractions = names.str.split('.').str[0]
names_v2 = names.str.split('.').str[1]
contractions.value_counts()
female_contrs = [' Miss', ' Mrs', ' Mme', ' Mlle', ' Ms', ' Lady', ' the Countess', ' Mme']
names = pd.DataFrame(names)
names['names_v2'] = names_v2.str.strip()
names['contractions'] = contractions
names['sex'] = 'male'
names.loc[names.contractions.isin(female_contrs), 'sex'] = 'female'
fem_names = names[names.sex == 'female']
names['row'] = 'ok'
names.loc[names.names_v2.str.contains('\\('), 'row'] = 'parse'
names_parse = names[names.row == 'parse']
names_v3 = names_parse.names_v2.str.split('(').str[1].str.split(')').str[0]
names_v4 = names.join(names_v3, rsuffix = '_left')
names_v4.loc[names_v4.row == 'parse', 'names_v2'] = names_v4.names_v2_left
names_v4 = names_v4.drop(columns=['names_v2_left', 'row'])
names_v4 = names_v4.rename(columns={'Name': 'names'})
names_v4['names_v3'] = names_v4.names_v2.str.split(' ').str[0]
names_v4[names_v4.sex == 'female'].names_v3.value_counts()


names_v3
Anna          15
Mary          14
Elizabeth     11
Margaret      10
Alice          6
              ..
Sylvia         1
Hilda          1
Margaretta     1
Miriam         1
Gerda          1
Name: count, Length: 177, dtype: int64

In [126]:
import datetime

In [176]:
def calc_angel(t: datetime.time) -> float:
    h = t.hour
    if h > 12:
        h = h - 12
    hour_angle = 0.5*h*60
    minute_angle = 6*t.minute
    angle = abs(hour_angle - minute_angle)
    return angle
calc_angel(datetime.time(11, 55))

0.0

In [178]:
def calc_angel_v2(t: datetime.time) -> float:
    h = t.hour
    if h > 12:
        h = h - 12
    hour_angle = 0.5*(h*60 + t.minute)
    minute_angle = 6*t.minute
    angel = abs(hour_angle - minute_angle)   
    return min(360 - angel, angel)
calc_angel_v2(datetime.time(3, 30))

75.0

# hw 2

In [ ]:
# Загрузите выборку из файла titanic.csv с помощью пакета Pandas.

# Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), возраст пассажира (Age) и его пол (Sex).

# Обратите внимание, что признак Sex имеет строковые значения.

# Выделите целевую переменную — она записана в столбце Survived.

# В данных есть пропущенные значения — например, для некоторых пассажиров неизвестен их возраст. 
# Такие записи при чтении их в pandas принимают значение nan. Найдите все объекты, у которых есть 
# пропущенные признаки, и удалите их из выборки. 

# Обучите решающее дерево с параметром random_state=241 и остальными параметрами по умолчанию 
# (речь идет о параметрах конструктора DecisionTreeСlassifier).

# Вычислите важности признаков и найдите два признака с наибольшей важностью. Их названия будут ответами для 
# данной задачи (в качестве ответа укажите названия признаков через запятую или пробел, порядок не важен). 

In [58]:
df_notna = df[df.Age.notna()]
df_encoded = pd.get_dummies(df_notna, columns=['Sex'])
df_encoded = df_encoded[['Sex_male','Sex_female']]
df_notna = df_notna[['Pclass', 'Fare', 'Age', 'Survived']]
X = df_notna.join(df_encoded).drop(columns=['Survived'])
print(df.shape, X.shape)
y = df_notna['Survived']

(891, 12) (714, 5)


In [59]:
clf = DecisionTreeClassifier(random_state=241)
clf.fit(X, y)
importances = clf.feature_importances_

In [60]:
imp_v1 = pd.DataFrame([X.columns, importances]).T
imp_v1.columns = ['feature', 'importance']
imp_v1.sort_values(by='importance', ascending=False)

,feature,importance
4,Sex_female,0.300512
1,Fare,0.300485
2,Age,0.253488
0,Pclass,0.145515
3,Sex_male,0.0
